# Herramienta para predecir accidentes

Con este proyecto vamos a intentar crear una herramienta que sea capaz de predecir accidentes durante el año 2020, entrenando nuestro modelo con datos de 2018 y 2019 proporcionados por la DGT. 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt


In [51]:
accidentes=pd.read_excel('C:/Users/aleja/Desktop/DataScience/Archivos_externos/2019_Accidentalidad.xlsx')

In [3]:
accidentes.shape

(51806, 14)

In [4]:
accidentes.columns

Index(['Nº  EXPEDIENTE', 'FECHA', 'HORA', 'CALLE', 'NÚMERO', 'DISTRITO',
       'TIPO ACCIDENTE', 'ESTADO METEREOLÓGICO', 'TIPO VEHÍCULO',
       'TIPO PERSONA', 'RANGO EDAD', 'SEXO', 'LESIVIDAD*',
       '* La correspondencia de los códigos se encuentra descrito en la estructura del fichero.'],
      dtype='object')

In [5]:
accidentes['RANGO EDAD'].unique()

array(['DE 25 A 29 AÑOS', 'DE 40 A 44 AÑOS', 'DE 35 A 39 AÑOS',
       'DE 30 A 34 AÑOS', 'DE 18 A 20 AÑOS', 'DE 55 A 59 AÑOS',
       'DESCONOCIDA', 'DE 50 A 54 AÑOS', 'DE 45 A 49 AÑOS',
       'DE 21 A 24 AÑOS', 'DE 65 A 69 AÑOS', 'DE 60 A 64 AÑOS',
       'DE 15 A 17 AÑOS', 'MAYOR DE 74 AÑOS', 'DE 70 A 74 AÑOS',
       'DE 10 A 14 AÑOS', 'DE 6 A 9 AÑOS', 'DE 0 A 5 AÑOS'], dtype=object)

In [52]:
edad_ordenado=['DE 0 A 5 AÑOS','DE 6 A 9 AÑOS','DE 10 A 14 AÑOS','DE 15 A 17 AÑOS','DE 18 A 20 AÑOS','DE 21 A 24 AÑOS','DE 25 A 29 AÑOS','DE 30 A 34 AÑOS','DE 35 A 39 AÑOS','DE 40 A 44 AÑOS','DE 45 A 49 AÑOS','DE 50 A 54 AÑOS','DE 55 A 59 AÑOS','DE 60 A 64 AÑOS','DE 65 A 69 AÑOS','DE 70 A 74 AÑOS','MAYOR DE 74 AÑOS']
accidentes['RANGO EDAD NUM']=accidentes.replace(edad_ordenado,[0,0,0,0,1,1,1,2,2,3,3,3,3,4,4,4,4])['RANGO EDAD']
accidentes.replace('DESCONOCIDA',np.nan,inplace=True)
accidentes[['RANGO EDAD', 'RANGO EDAD NUM']]

,RANGO EDAD,RANGO EDAD NUM
0,DE 25 A 29 AÑOS,1.0
1,DE 40 A 44 AÑOS,3.0
2,DE 35 A 39 AÑOS,2.0
3,DE 40 A 44 AÑOS,3.0
4,DE 30 A 34 AÑOS,2.0
...,...,...
51801,DE 18 A 20 AÑOS,1.0
51802,DE 18 A 20 AÑOS,1.0
51803,NaN,NaN
51804,DE 21 A 24 AÑOS,1.0


In [7]:
accidentes['TIPO VEHÍCULO'].unique()

array(['Ciclomotor', 'Turismo', 'Todo terreno', 'Bicicleta',
       'Motocicleta hasta 125cc', 'Motocicleta > 125cc', 'Autobús',
       'Furgoneta', 'Camión rígido', nan, 'Tractocamión',
       'Maquinaria de obras', 'Vehículo articulado',
       'Cuadriciclo no ligero', 'Autobús articulado',
       'Otros vehículos con motor', 'Autocaravana', 'Ciclo',
       'Cuadriciclo ligero', 'Semiremolque', 'Microbús <= 17 plazas',
       'Sin especificar', 'Remolque', 'Tranvía', 'Caravana',
       'VMU eléctrico', 'Otros vehículos sin motor', 'Patinete',
       'Bicicleta EPAC (pedaleo asistido)', 'Autobus EMT'], dtype=object)

In [53]:
accidentes['TIPO VEHÍCULO NUM']=accidentes.replace(np.delete(accidentes['TIPO VEHÍCULO'].unique(),9),[1,2,2,0,1,1,3,2,3,3,4,4,1,3,4,3,0,1,4,3,0,4,4,3,0,0,0,0,3])['TIPO VEHÍCULO']

In [10]:
accidentes['TIPO ACCIDENTE'].unique()

array(['Caída', 'Colisión fronto-lateral', 'Atropello a persona',
       'Choque contra obstáculo fijo', 'Alcance', 'Vuelco',
       'Atropello a animal', 'Otro', 'Colisión lateral',
       'Colisión frontal', 'Colisión múltiple', 'Solo salida de la vía',
       'Despeñamiento', nan], dtype=object)

In [54]:
accidentes['TIPO ACCIDENTE NUM']=accidentes.replace(np.delete(accidentes['TIPO ACCIDENTE'].unique(),13),[1,2,3,4,4,1,3,0,2,2,2,0,1])['TIPO ACCIDENTE']

In [55]:
accidentes['MES']=pd.to_datetime(accidentes['FECHA']).dt.strftime('%m').astype('int')

In [56]:
rango=[]
for i in accidentes.HORA:
    if i >= datetime.strptime('06:00','%H:%M').time() and i < datetime.strptime('12:00','%H:%M').time():
        rango.append(0)
    elif i >= datetime.strptime('12:00','%H:%M').time() and i < datetime.strptime('20:00','%H:%M').time():
        rango.append(1)
    else:
        rango.append(2)
    
accidentes['RANGO HORARIO']=rango

In [57]:
columnas=['DISTRITO',
        'ESTADO METEREOLÓGICO',
       'TIPO PERSONA','SEXO']
for i in columnas:
    accidentes[i + ' NUM']=accidentes.replace(accidentes[i].dropna().unique(),np.arange(len(accidentes[i].dropna().unique())))[i]

In [18]:
accidentes.columns

Index(['Nº  EXPEDIENTE', 'FECHA', 'HORA', 'CALLE', 'NÚMERO', 'DISTRITO',
       'TIPO ACCIDENTE', 'ESTADO METEREOLÓGICO', 'TIPO VEHÍCULO',
       'TIPO PERSONA', 'RANGO EDAD', 'SEXO', 'LESIVIDAD*',
       '* La correspondencia de los códigos se encuentra descrito en la estructura del fichero.',
       'RANGO EDAD NUM', 'TIPO ACCIDENTE NUM', 'TIPO VEHÍCULO NUM', 'MES',
       'RANGO HORARIO', 'DISTRITO NUM', 'ESTADO METEREOLÓGICO NUM',
       'TIPO PERSONA NUM', 'SEXO NUM'],
      dtype='object')

In [58]:
accidentes['SEXO NUM'].unique()

array([ 0.,  1., nan])

In [59]:
accidentes['LESIVIDAD*'].unique()

array([ 1., 14.,  3.,  7., nan,  2.,  5.,  6.,  4., 77.])

In [60]:
accidentes['LESIVIDAD*'].replace(np.nan,0,inplace=True)

In [61]:
accidentes_num=accidentes[['MES', 'RANGO HORARIO', 'RANGO EDAD NUM', 'DISTRITO NUM',
       'TIPO ACCIDENTE NUM', 'ESTADO METEREOLÓGICO NUM', 'TIPO VEHÍCULO NUM',
       'TIPO PERSONA NUM', 'SEXO NUM','LESIVIDAD*']]
accidentes_num.head()

,MES,RANGO HORARIO,RANGO EDAD NUM,DISTRITO NUM,TIPO ACCIDENTE NUM,ESTADO METEREOLÓGICO NUM,TIPO VEHÍCULO NUM,TIPO PERSONA NUM,SEXO NUM,LESIVIDAD*
0,1,2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,1,2,3.0,1.0,2.0,0.0,2.0,0.0,1.0,14.0
2,1,2,2.0,1.0,2.0,0.0,1.0,0.0,0.0,3.0
3,1,2,3.0,2.0,2.0,0.0,2.0,0.0,0.0,14.0
4,1,2,2.0,2.0,2.0,0.0,2.0,0.0,1.0,7.0


## Estadísticas

**1.**	Picos en el año donde hay más accidentes (graficar)

In [ ]:
sns.countplot('MES',data=accidentes)

**2.**	5 distritos con más accidentes (graficar)

In [ ]:
distrito=accidentes.groupby('DISTRITO',as_index=False)['Nº  EXPEDIENTE'].count().sort_values(by='Nº  EXPEDIENTE',ascending=False)[0:5]
distrito

In [ ]:
sns.barplot(distrito['DISTRITO'],distrito['Nº  EXPEDIENTE'])

**3.**	Horario con más accidentes (categorizar por: mañana, tarde, noche)

In [ ]:
accidentes['RANGO HORARIO'].value_counts()

**4.**	Identificar por distritos tipos de accidentes (graficar)

In [ ]:
for num,i in enumerate(accidentes.DISTRITO.unique()):
    fig =plt.figure(num,figsize=(7,5))
    plt.title(i)
    sns.countplot(y='TIPO ACCIDENTE',data=accidentes[accidentes['DISTRITO']== i])

In [ ]:
distrito[distrito['DISTRITO']== 'CENTRO']

**5.**	Identificar los 3 tramos de edad con más accidentes (graficar)

**6.**	3 tipos de accidentes que más suceden por distrito

**7.**	Graficar en orden descendente los días de la semana con más accidentes 

**8.**	Tipo de accidente más común en cada distrito

**9.**	"Lugar de accidente” con más accidentes por “tipo de accidente”

**10.**	 “Tipo de vehículo” con más accidentes. 

**11.** Número de accidentes en función del tipo de vehículo

**12.** Tipo de accidente en función de las condiciones atmosféricas y de la calzada

**13.** Distrito con más accidentes y que tengan algún grado de lesividad, teniendo un estado meteorológico despejado

**14.** Porcentaje de accidentes según diferentes condiciones meteorológicas y de asfalto, en función del tipo de vehículo

**15.** Grado de lesividad según tipo de accidente 

**16.** Horarios más comunes para accidentes según distrito.

## Preguntas a responder

**1.**	¿Relación entre tramo de edad y sexo? 

In [84]:
accidentes_num[['RANGO EDAD NUM','SEXO NUM']].corr()

,RANGO EDAD NUM,SEXO NUM
RANGO EDAD NUM,1.000000,-0.086067
SEXO NUM,-0.086067,1.000000


**2.**	¿Cuál es la correlación entre los atributos?

In [85]:
accidentes_num.corr()

,MES,RANGO HORARIO,RANGO EDAD NUM,DISTRITO NUM,TIPO ACCIDENTE NUM,ESTADO METEREOLÓGICO NUM,TIPO VEHÍCULO NUM,TIPO PERSONA NUM,SEXO NUM,LESIVIDAD*
MES,1.000000,-0.000472,-0.001532,0.008606,-0.003591,0.114230,-0.015974,-0.003835,-0.001709,-0.013342
RANGO HORARIO,-0.000472,1.000000,-0.079613,-0.037776,0.023490,0.041468,-0.033055,0.026792,-0.022967,-0.056408
RANGO EDAD NUM,-0.001532,-0.079613,1.000000,0.006435,0.009552,-0.025542,0.126589,-0.154787,-0.086067,-0.049870
DISTRITO NUM,0.008606,-0.037776,0.006435,1.000000,0.028825,0.017466,0.052393,-0.008419,0.029155,0.009769
TIPO ACCIDENTE NUM,-0.003591,0.023490,0.009552,0.028825,1.000000,-0.019269,0.089062,0.045568,0.006967,-0.018308
ESTADO METEREOLÓGICO NUM,0.114230,0.041468,-0.025542,0.017466,-0.019269,1.000000,-0.026409,0.017667,0.010936,0.022503
TIPO VEHÍCULO NUM,-0.015974,-0.033055,0.126589,0.052393,0.089062,-0.026409,1.000000,0.057951,0.025600,0.029789
TIPO PERSONA NUM,-0.003835,0.026792,-0.154787,-0.008419,0.045568,0.017667,0.057951,1.000000,0.286797,0.038157
SEXO NUM,-0.001709,-0.022967,-0.086067,0.029155,0.006967,0.010936,0.025600,0.286797,1.000000,0.065712
LESIVIDAD*,-0.013342,-0.056408,-0.049870,0.009769,-0.018308,0.022503,0.029789,0.038157,0.065712,1.000000


**3.**	¿Los días feriados tienen relación con los accidentes?

**4.**	¿Cuál es el día con más accidentes y porque crees que sucedió?

In [86]:
accidentes.groupby('FECHA')['Nº  EXPEDIENTE'].count().idxmax()

'05/04/2019'

In [87]:
accidentes[accidentes['FECHA']=='05/04/2019'].groupby('ESTADO METEREOLÓGICO')['Nº  EXPEDIENTE'].count().sort_values(ascending=False)

ESTADO METEREOLÓGICO
Lluvia débil      154
Despejado          56
Nublado            38
LLuvia intensa      7
Granizando          7
Se desconoce        6
Name: Nº  EXPEDIENTE, dtype: int64

**5.**	¿Cuál es la frecuencia de accidentes en cada distrito?

In [88]:
accidentes.groupby('DISTRITO')['Nº  EXPEDIENTE'].count().sort_values(ascending=False)

DISTRITO
SALAMANCA              4078
PUENTE DE VALLECAS     4063
CHAMARTÍN              3891
CARABANCHEL            3365
CIUDAD LINEAL          3362
SAN BLAS-CANILLEJAS    2797
CENTRO                 2752
MONCLOA-ARAVACA        2688
CHAMBERÍ               2680
RETIRO                 2671
LATINA                 2528
FUENCARRAL-EL PARDO    2468
TETUÁN                 2382
ARGANZUELA             2271
HORTALEZA              2025
USERA                  1794
MORATALAZ              1530
VILLAVERDE             1421
VILLA DE VALLECAS      1299
BARAJAS                1071
VICÁLVARO               665
Name: Nº  EXPEDIENTE, dtype: int64

**6.**	¿Qué probabilidad tiene una moto de sufrir un accidente y de que tipo sería, en función de las condiciones atmosféricas y de la calzada?

In [89]:
accidentes['TIPO VEHÍCULO'].unique()

array(['Ciclomotor', 'Turismo', 'Todo terreno', 'Bicicleta',
       'Motocicleta hasta 125cc', 'Motocicleta > 125cc', 'Autobús',
       'Furgoneta', 'Camión rígido', nan, 'Tractocamión',
       'Maquinaria de obras', 'Vehículo articulado',
       'Cuadriciclo no ligero', 'Autobús articulado',
       'Otros vehículos con motor', 'Autocaravana', 'Ciclo',
       'Cuadriciclo ligero', 'Semiremolque', 'Microbús <= 17 plazas',
       'Sin especificar', 'Remolque', 'Tranvía', 'Caravana',
       'VMU eléctrico', 'Otros vehículos sin motor', 'Patinete',
       'Bicicleta EPAC (pedaleo asistido)', 'Autobus EMT'], dtype=object)

In [90]:
motos=['Ciclomotor','Motocicleta hasta 125cc','Motocicleta > 125cc']
accidentes_motos=accidentes[accidentes['TIPO VEHÍCULO'].isin(motos)]
print('Probabilidad de que una moto tenga un accidente: ' + str(len(accidentes_motos)/len(accidentes)))
accidentes_motos.groupby('ESTADO METEREOLÓGICO')['Nº  EXPEDIENTE'].count().sort_values(ascending=False)

Probabilidad de que una moto tenga un accidente: 0.13251360846234028


ESTADO METEREOLÓGICO
Despejado         5528
Lluvia débil       587
Nublado            262
Se desconoce       109
LLuvia intensa      96
Granizando           2
Nevando              1
Name: Nº  EXPEDIENTE, dtype: int64

**7.**	¿Cuál es el tipo de accidente más común de acuerdo al tipo de vehículo?

In [91]:
for i in accidentes['TIPO VEHÍCULO'].unique():
    try:
        print(str(i) + ': ' + str(accidentes[accidentes['TIPO VEHÍCULO']==i].groupby('TIPO ACCIDENTE')['Nº  EXPEDIENTE'].count().idxmax()))
    except:
        print(str(i) + ': NONE') 
        

Ciclomotor: Caída
Turismo: Alcance
Todo terreno: Alcance
Bicicleta: Caída
Motocicleta hasta 125cc: Caída
Motocicleta > 125cc: Colisión fronto-lateral
Autobús: Colisión fronto-lateral
Furgoneta: Alcance
Camión rígido: Colisión lateral
nan: NONE
Tractocamión: Colisión lateral
Maquinaria de obras: Colisión lateral
Vehículo articulado: Colisión lateral
Cuadriciclo no ligero: Atropello a persona
Autobús articulado: Colisión fronto-lateral
Otros vehículos con motor: Caída
Autocaravana: Colisión fronto-lateral
Ciclo: Caída
Cuadriciclo ligero: Colisión fronto-lateral
Semiremolque: Colisión fronto-lateral
Microbús <= 17 plazas: Alcance
Sin especificar: Colisión fronto-lateral
Remolque: Colisión lateral
Tranvía: Alcance
Caravana: Choque contra obstáculo fijo
VMU eléctrico: Caída
Otros vehículos sin motor: Atropello a persona
Patinete: Caída
Bicicleta EPAC (pedaleo asistido): Caída
Autobus EMT: Atropello a persona


**8.**	¿Aumenta el número de accidentes en función del estado del asfalto y del clima?

In [92]:
accidentes.groupby('ESTADO METEREOLÓGICO')['Nº  EXPEDIENTE'].count().sort_values(ascending=False)

ESTADO METEREOLÓGICO
Despejado         39984
Lluvia débil       3465
Nublado            1862
Se desconoce        695
LLuvia intensa      652
Granizando           14
Nevando               2
Name: Nº  EXPEDIENTE, dtype: int64

**9.**	¿Cuál es el horario y que rango de edad tiene cuando se producen más accidentes?

In [93]:
accidentes.groupby('RANGO HORARIO')['Nº  EXPEDIENTE'].count().sort_values(ascending=False)

RANGO HORARIO
1    25559
2    14393
0    11854
Name: Nº  EXPEDIENTE, dtype: int64

In [94]:
accidentes[accidentes['RANGO HORARIO']==1].groupby('RANGO EDAD')['Nº  EXPEDIENTE'].count().sort_values(ascending=False)

RANGO EDAD
DE 40 A 44 AÑOS     3040
DE 35 A 39 AÑOS     2726
DE 45 A 49 AÑOS     2634
DE 30 A 34 AÑOS     2608
DE 25 A 29 AÑOS     2602
DE 50 A 54 AÑOS     2189
DE 55 A 59 AÑOS     1710
DE 21 A 24 AÑOS     1408
DE 60 A 64 AÑOS     1130
MAYOR DE 74 AÑOS     741
DE 18 A 20 AÑOS      640
DE 65 A 69 AÑOS      569
DE 70 A 74 AÑOS      431
DE 0 A 5 AÑOS        319
DE 10 A 14 AÑOS      305
DE 6 A 9 AÑOS        253
DE 15 A 17 AÑOS      196
Name: Nº  EXPEDIENTE, dtype: int64

**10.**	¿Cuáles son los horarios donde debería de haber más agentes de transito y en qué lugares específicamente?      

In [95]:
accidentes.groupby(['DISTRITO','RANGO HORARIO'])['Nº  EXPEDIENTE'].count().sort_values(ascending=False)[0:15]

DISTRITO             RANGO HORARIO
PUENTE DE VALLECAS   1                2118
SALAMANCA            1                2090
CHAMARTÍN            1                1968
CIUDAD LINEAL        1                1620
CARABANCHEL          1                1601
SAN BLAS-CANILLEJAS  1                1543
MONCLOA-ARAVACA      1                1420
RETIRO               1                1391
CHAMBERÍ             1                1261
LATINA               1                1221
FUENCARRAL-EL PARDO  1                1190
TETUÁN               1                1177
SALAMANCA            2                1165
PUENTE DE VALLECAS   2                1152
CENTRO               1                1133
Name: Nº  EXPEDIENTE, dtype: int64

## Predicción

In [62]:
from sklearn import neighbors
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [63]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputer.fit(accidentes_num)
accidentes_num_imp = imputer.transform(accidentes_num)
accidentes_num = pd.DataFrame(accidentes_num_imp, columns = accidentes_num.columns)
accidentes_num.head(4)

,MES,RANGO HORARIO,RANGO EDAD NUM,DISTRITO NUM,TIPO ACCIDENTE NUM,ESTADO METEREOLÓGICO NUM,TIPO VEHÍCULO NUM,TIPO PERSONA NUM,SEXO NUM,LESIVIDAD*
0,1.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,1.0,2.0,3.0,1.0,2.0,0.0,2.0,0.0,1.0,14.0
2,1.0,2.0,2.0,1.0,2.0,0.0,1.0,0.0,0.0,3.0
3,1.0,2.0,3.0,2.0,2.0,0.0,2.0,0.0,0.0,14.0


**1.**	Predecir el tramo de edad 

In [64]:
X=np.asarray(accidentes_num.drop('RANGO EDAD NUM', axis=1))
y=np.asarray(accidentes_num['RANGO EDAD NUM'])
clf = neighbors.KNeighborsClassifier(3, weights="distance")
scores = cross_val_score(clf, X, y, cv=10, scoring = make_scorer(f1_score,average='macro'))
print(scores)
print("*****"*10)
print("F1-score: ", scores.mean())

[0.24177119 0.26037182 0.2500277  0.24394484 0.25861665 0.26345092
 0.25177514 0.2454727  0.24543372 0.25637862]
**************************************************
F1-score:  0.25172433022804824


**2.**	Predecir el tipo de accidente 

In [65]:
X=np.asarray(accidentes_num.drop('TIPO ACCIDENTE NUM', axis=1))
y=np.asarray(accidentes_num['TIPO ACCIDENTE NUM'])

In [66]:
clf = neighbors.KNeighborsClassifier(3, weights="distance")
scores = cross_val_score(clf, X, y, cv=10, scoring = make_scorer(f1_score,average='macro'))
print(scores)
print("*****"*10)
print("F1-score: ", scores.mean())

[0.30432385 0.31718821 0.31322759 0.30182127 0.30895149 0.3253664
 0.30775519 0.29922802 0.3027084  0.29166353]
**************************************************
F1-score:  0.30722339314757885


In [ ]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
loo.get_n_splits(X)
yhat=[]
for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    #CLASIFICADOR 3NN
    clf = neighbors.KNeighborsClassifier(3, weights="distance")
    clf.fit(X_train, y_train)
    yhat1 = clf.predict(X_test)
    yhat.append(yhat1)
print("F1-score: ", f1_score(y,yhat))

**3.**	Predecir el día del accidente 

**4.**	Predecir el mes de un accidente

In [67]:
X=np.asarray(accidentes_num.drop('MES', axis=1))
y=np.asarray(accidentes_num['MES'])
clf = neighbors.KNeighborsClassifier(3, weights="distance")
scores = cross_val_score(clf, X, y, cv=10, scoring = make_scorer(f1_score,average='macro'))
print(scores)
print("*****"*10)
print("F1-score: ", scores.mean())

[0.07658023 0.07233812 0.07521269 0.08286914 0.07846741 0.08759712
 0.09786511 0.0748617  0.08132082 0.08834817]
**************************************************
F1-score:  0.08154605090061304


**5.**	Predecir el horario del accidente (categorizar por: mañana, tarde, noche)

In [68]:
X=np.asarray(accidentes_num.drop('RANGO HORARIO', axis=1))
y=np.asarray(accidentes_num['RANGO HORARIO'])
clf = neighbors.KNeighborsClassifier(3, weights="distance")
scores = cross_val_score(clf, X, y, cv=10, scoring = make_scorer(f1_score,average='macro'))
print(scores)
print("*****"*10)
print("F1-score: ", scores.mean())

[0.34175346 0.33473948 0.32704531 0.32410544 0.34122845 0.33771562
 0.33917775 0.33506018 0.33694016 0.36128743]
**************************************************
F1-score:  0.33790532778813104


**6.**	Proponer

In [69]:
X=np.asarray(accidentes_num.drop('LESIVIDAD*', axis=1))
y=np.asarray(accidentes_num['LESIVIDAD*'])
clf = neighbors.KNeighborsClassifier(3, weights="distance")
scores = cross_val_score(clf, X, y, cv=10, scoring = make_scorer(f1_score,average='macro'))
print(scores)
print("*****"*10)
print("F1-score: ", scores.mean())

C:\Users\aleja\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
C:\Users\aleja\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\aleja\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[0.13941075 0.1429253  0.14335092 0.16070799 0.15859546 0.15688918
 0.16806628 0.15417201 0.16069594 0.14576861]
**************************************************
F1-score:  0.1530582451282619


## Clustering

**1.**	Utilizar “tramo de edad” y “horario del accidente” para hacer clustering 

In [ ]:
accidentes_num.columns

In [ ]:
plt.scatter(accidentes_num['RANGO EDAD NUM'], accidentes_num['RANGO HORARIO'], marker='.')

In [ ]:
from sklearn.cluster import KMeans 
k_means = KMeans(init = "k-means++", n_clusters = 3, n_init = 12)

In [ ]:
k_means.fit(np.asarray(accidentes_num[['RANGO EDAD NUM','RANGO HORARIO']]))

In [ ]:
k_means_labels = k_means.labels_
k_means_cluster_centers = k_means.cluster_centers_
print(k_means_labels)
print(k_means_cluster_centers)

**2.**	Utilizar “Dia de la semana” y “tramo de edad” para hacer clustering

In [ ]:
plt.scatter(accidentes_num['MES'], accidentes_num['RANGO EDAD NUM'], marker='.')

In [ ]:
from sklearn.cluster import KMeans 
k_means = KMeans(init = "k-means++", n_clusters = 4, n_init = 12)

In [ ]:
k_means.fit(np.asarray(X[['MES','RANGO EDAD NUM']]))

In [ ]:
k_means_labels = k_means.labels_
k_means_cluster_centers = k_means.cluster_centers_
print(k_means_labels)
print(k_means_cluster_centers)

**3.**	Proponer

## Series de tiempo con RNN

In [72]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN

Using TensorFlow backend.


In [74]:
len(accidentes_num.columns)

10

**1.**	Predecir el tipo de accidente respecto al tiempo. (graficar)

In [76]:
len(accidentes_num['TIPO ACCIDENTE NUM'].unique())

5

In [81]:
X=np.asarray(accidentes_num.drop('TIPO ACCIDENTE NUM', axis=1))
y=np.asarray(accidentes_num['TIPO ACCIDENTE NUM'])
trainX, testX, trainy, testy = train_test_split( X, y, test_size=0.3, random_state=4)

In [ ]:
#Las RNN requieren de un valor de paso que contiene n número de elementos como secuencia de entrada. 
#lo definimos como un 'step'.
# convert into dataset matrix
step = 365

def convertToMatrix(data, step):
    X, Y =[], []
    for i in range(len(data)-step):
        d=i+step  
        X.append(data[i:d,])
        Y.append(data[d,])
    return np.array(X), np.array(Y)

# add step elements into train and test
test = np.append(test,np.repeat(test[-1,],step))
train = np.append(train,np.repeat(train[-1,],step))

trainX,trainY =convertToMatrix(train,step)
testX,testY =convertToMatrix(test,step)
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [82]:
# MODELO
model = Sequential()
model.add(SimpleRNN(9, input_shape=(1,9), activation="relu", unroll=True))
model.add(Dense(8, activation="relu")) 
model.add(Dense(5,activation='softmax'))
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (None, 9)                 171       
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 80        
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 45        
Total params: 296
Trainable params: 296
Non-trainable params: 0
_________________________________________________________________


In [83]:
model.fit(trainX,trainy, epochs=100, batch_size=64, verbose=2)
trainPredict = model.predict(trainX)
testPredict= model.predict(testX)

predicted=np.concatenate((trainPredict,testPredict),axis=0,)
trainScore = model.evaluate(trainX, trainY, verbose=0)

ValueError: Error when checking input: expected simple_rnn_2_input to have 3 dimensions, but got array with shape (36264, 9)

**2.**	Predecir el tramo de edad respecto al tiempo. (graficar)**

**3.**	Predecir “CPFA Nieve” respecto al tiempo. (graficar)

**4.**	Proponer

## Comparar 2018 con 2019 para estimar y planificar 2020 

**1.**	Identificar razones de cambio

**2.**	Estimar que zonas tendrán más accidentes durante el 2020

## Proponer un producto viable para la DGT

## Describir el Notebook para subirlo a GitHub

## Crear un resumen, explicación de cada resultado, conclusiones de todo el trabajo y trabajo a futuro.